# Rabi experiment

In [ ]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output

from qubecalib import QubeCalib
from qubecalib.neopulse import (
    Blank,
    Capture,
    Flushleft,
    Flushright,
    RaisedCosFlatTop,
    Rectangle,
    Sequence,
)

In [ ]:
%matplotlib inline

In [ ]:
# load the system settings
qc = QubeCalib("./system_settings.json")

In [ ]:
# check the settings
qc.system_config_database.asdict()

In [ ]:
box_list = [
    "A25",
    "A73",
]

boxes = {}
for box_name in box_list:
    box = qc.create_box(box_name, reconnect=False)
    if not all(box.link_status().values()):
        box.relinkup(use_204b=False, background_noise_threshold=400)
    box.reconnect()
    boxes[box_name] = box
    print(box_name, box.link_status())

In [ ]:
for box in boxes.values():
    print(box.dump_box())

In [ ]:
def rabi_sequence(T: int) -> Sequence:
    """
    Rabi sequence with a single target.

    Parameters
    ----------
    T : int
        Duration of the rectangle pulse in ns.

    Returns
    -------
    Sequence
        The rabi sequence.
    """
    control_pulse = Rectangle(duration=T, amplitude=0.01)
    readout_pulse = RaisedCosFlatTop(duration=1024, amplitude=0.1, rise_time=128)

    with Sequence() as seq:
        with Flushright():
            control_pulse.scaled(8).target("CQ16")
            control_pulse.scaled(3).target("CQ52")
            Blank(duration=1024).target()
        with Flushleft():
            readout_pulse.scaled(2).target("RQ16")
            readout_pulse.scaled(1).target("RQ52")
            Capture(duration=2 * 1024).target("RQ16", "RQ52")
    return seq


# add rabi sequences to the queue
sweep_range = np.arange(0, 200, 10)
for T in sweep_range:
    qc.add_sequence(rabi_sequence(T))


# execute the sequences
signals = defaultdict(list)
for status, data, config in qc.step_execute(
    repeats=3000,
    interval=150 * 1024,
):
    for target, iqs in data.items():
        iq = iqs[0].squeeze().mean()
        signals[target].append(iq)
    clear_output(True)
    fig, axs = plt.subplots(len(signals), 1, figsize=(5, 5 * len(signals)))
    for i, (target, signal) in enumerate(signals.items()):
        axs[i].scatter(np.real(signal), np.imag(signal), label=target)
        axs[i].set_title(target)
        axs[i].grid()
    plt.show()

In [ ]:
fig, axs = plt.subplots(len(signals), 1, figsize=(6, 4 * len(signals)))
for i, (target, signal) in enumerate(signals.items()):
    axs[i].plot(np.real(signal), "o-", label="I")
    axs[i].plot(np.imag(signal), "o-", label="Q")
    axs[i].set_title(target)
    axs[i].legend()
    axs[i].grid()